In [3]:
# !pip install transforms3d --break-system-packages

In [6]:
import numpy as np
import transforms3d
from geometry_msgs.msg import Pose
import rclpy
from rclpy.node import Node
from geometry_msgs.msg import Pose
from sensor_msgs.msg import JointState
from ikpy.chain import Chain
import numpy as np
import time
import transforms3d

In [ ]:

def apriltag_to_pose(r):
    t = r.pose_t.flatten()           # 태그의 위치 벡터 [x, y, z]
    R = r.pose_R                     # 태그의 회전 행렬 (3x3)

    # transforms3d의 mat2quat 함수는 (w, x, y, z) 순서로 반환합니다.
    quat_wxyz = transforms3d.quaternions.mat2quat(R)
    # ROS Pose 메시지는 (x, y, z, w) 순서이므로 변환 필요
    quat = [quat_wxyz[1], quat_wxyz[2], quat_wxyz[3], quat_wxyz[0]]

    pose = Pose()
    pose.position.x = t[0]
    pose.position.y = t[1]
    pose.position.z = t[2]
    pose.orientation.x = quat[0]
    pose.orientation.y = quat[1]
    pose.orientation.z = quat[2]
    pose.orientation.w = quat[3]
    return pose


In [ ]:

# 예시: AprilTag 검출 결과에서 얻은 회전행렬(R)과 위치벡터(t)
R = np.array([
    [0, -1, 0],
    [1,  0, 0],
    [0,  0, 1]
])
t = np.array([0.1, 0.2, 0.3])

# 회전행렬을 쿼터니언으로 변환 (transforms3d는 (w, x, y, z) 순서로 반환)
quat_wxyz = transforms3d.quaternions.mat2quat(R)
# ROS Pose 메시지와 동일하게 (x, y, z, w) 순서로 재정렬
quat_xyzw = [quat_wxyz[1], quat_wxyz[2], quat_wxyz[3], quat_wxyz[0]]

# 결과 출력
print("Position (t):")
print(f"x = {t[0]:.3f}, y = {t[1]:.3f}, z = {t[2]:.3f}")

print("\nQuaternion (x, y, z, w):")
print(f"x = {quat_xyzw[0]:.4f}")
print(f"y = {quat_xyzw[1]:.4f}")
print(f"z = {quat_xyzw[2]:.4f}")
print(f"w = {quat_xyzw[3]:.4f}")


Position (t):
x = 0.100, y = 0.200, z = 0.300

Quaternion (x, y, z, w):
x = 0.0000
y = 0.0000
z = 0.7071
w = 0.7071


In [4]:
R = np.array([
    [0, -1, 0],
    [1,  0, 0],
    [0,  0, 1]
])
t = np.array([0.1, 0.2, 0.3])

In [5]:
# 회전행렬 → 쿼터니언 변환 (w, x, y, z 순서)
quat_wxyz = transforms3d.quaternions.mat2quat(R)
quat_xyzw = [quat_wxyz[1], quat_wxyz[2], quat_wxyz[3], quat_wxyz[0]] 
print(quat_xyzw)
# publish_pose(t, quat_xyzw) apriltag에서 publish 하는 값


[0.0, 0.0, 0.7071067811865475, 0.7071067811865475]


In [7]:
quat_wxyz

array([0.70710678, 0.        , 0.        , 0.70710678])

In [8]:
# Pose → 위치/회전행렬
position = t
# 쿼터니언 (x, y, z, w) → 회전행렬
quat_wxyz 
rot = transforms3d.quaternions.quat2mat(quat_wxyz)
print(rot)

[[ 0. -1.  0.]
 [ 1.  0.  0.]
 [ 0.  0.  1.]]


In [ ]:
# IKPy 역기구학
angles = chain.inverse_kinematics(
    position,
    target_orientation=rot, 
)

In [ ]:
import numpy as np
from scipy.spatial.transform import Rotation as R

# 프레임 연결 순서 정의
CHAIN = [
    # ('g_base', 'link1'),
    ('link1', 'link2'),
    ('link2', 'link3'),
    ('link3', 'link4'),
    ('link4', 'link5'),
    ('link5', 'link6'),
    ('link6', 'link6_flange'),
    # ('link6_flange', 'jetcocam'),
    ('jetcocam', 'tag36h11:4')
]

# 각 변환 정보 (translation, quaternion)
tf_dict = {
    ('g_base', 'link1'): (np.array([0.0, 0.0, 0.0]), np.array([0.0, 0.0, 0.0, 1.0])),
    ('link1', 'link2'): (np.array([0.0, 0.0, 0.13156]), np.array([0.0, 0.0, 0.0, 1.0])),
    ('link2', 'link3'): (np.array([0.0, 0.0, 0.0]), np.array([0.5000018366025517, 0.4999999999966269, -0.4999999999966269, 0.49999816339744835])),
    ('link3', 'link4'): (np.array([-0.1104, 0.0, 0.0]), np.array([0.0, 0.0, 0.0, 1.0])),
    ('link4', 'link5'): (np.array([-0.096, 0.0, 0.06462]), np.array([0.0, 0.0, -0.7071080798594737, 0.7071054825112364])),
    ('link5', 'link6'): (np.array([0.0, -0.07318, 0.0]), np.array([0.4999999999966269, -0.4999999999966269, 0.5000018366025517, 0.49999816339744835])),
    ('link6', 'link6_flange'): (np.array([0.0, 0.0456, 0.0]), np.array([-0.7071080798594738, 0.0, 0.0, 0.7071054825112363])),
    # 아래 변환이 누락된 경우 단위 변환(Identity)로 가정
    ('link6_flange', 'jetcocam'): (np.array([0.0, 0.0, 0.0]), np.array([0.0, 0.0, 0.0, 1.0])),
    ('jetcocam', 'tag36h11:4'): (np.array([0.03472433263398279, -0.00023349412724901148, 0.7047155805245154]),
                                 np.array([0.9263294433436424, -0.36284512666426993, 0.042356020613716655, -0.0919953475368079]))
}

# 누적 변환 계산
T = np.eye(4)
for parent, child in CHAIN:
    t, q = tf_dict[(parent, child)]
    T_i = np.eye(4)
    T_i[:3, :3] = R.from_quat(q).as_matrix()
    T_i[:3, 3] = t
    T = T @ T_i

# 최종 위치(pos)와 쿼터니언(quat) 추출
pos = T[:3, 3]
quat = R.from_matrix(T[:3, :3]).as_quat()

print('최종 위치 (g_base → tag36h11:4):', pos)
print('최종 쿼터니언 (g_base → tag36h11:4):', quat)


최종 위치 (g_base → tag36h11:4): [ 0.75031561 -0.09934536  0.41137054]
최종 쿼터니언 (g_base → tag36h11:4): [-0.34891859  0.66940542  0.21456834 -0.61976821]


In [12]:
import numpy as np
from scipy.spatial.transform import Rotation as R

# 프레임 연결 순서 정의
CHAIN = [
    # ('map', 'g_base'),0
    ('g_base', 'link1'),
    ('link1', 'link2'),
    ('link2', 'link3'),
    ('link3', 'link4'),
    ('link4', 'link5'),
    ('link5', 'link6'),
    ('link6', 'link6_flange'),
    ('link6_flange', 'gripper_base'),
    ('gripper_base', 'jetcocam'),
    ('jetcocam', 'tag36h11:4')
]

# 각 변환 정보 (translation, quaternion)
tf_dict = {
    ('map', 'g_base'): (np.array([0.0, 0.0, 0.0]), np.array([0.0, 0.0, 0.0, 1.0])),
    ('g_base', 'link1'): (np.array([0.0, 0.0, 0.0]), np.array([0.0, 0.0, 0.0, 1.0])),
    ('link1', 'link2'): (np.array([0.0, 0.0, 0.13156]), np.array([0.0, 0.0, 0.0, 1.0])),
    ('link2', 'link3'): (np.array([0.0, 0.0, 0.0]), np.array([0.5000018366025517, 0.4999999999966269, -0.4999999999966269, 0.49999816339744835])),
    ('link3', 'link4'): (np.array([-0.1104, 0.0, 0.0]), np.array([0.0, 0.0, 0.0, 1.0])),
    ('link4', 'link5'): (np.array([-0.096, 0.0, 0.06462]), np.array([0.0, 0.0, -0.7071080798594737, 0.7071054825112364])),
    ('link5', 'link6'): (np.array([0.0, -0.07318, 0.0]), np.array([0.4999999999966269, -0.4999999999966269, 0.5000018366025517, 0.49999816339744835])),
    ('link6', 'link6_flange'): (np.array([0.0, 0.0456, 0.0]), np.array([-0.7071080798594738, 0.0, 0.0, 0.7071054825112363])),
    ('link6_flange', 'gripper_base'): (np.array([0.0, 0.0, 0.034]), np.array([-0.2717039124397632, 0.65595638159753, 0.6505970827175301, -0.269484035456352])),
    ('gripper_base', 'jetcocam'): (np.array([0.0, 0.0, -0.048]), np.array([2.597353007557415e-06, 0.707105482506466, 0.7071080798547033, -2.5973434669646147e-06])),
    ('jetcocam', 'tag36h11:4'): (np.array([0.03472433263398279, -0.00023349412724901148, 0.7047155805245154]), np.array([0.9263294433436424, -0.36284512666426993, 0.042356020613716655, -0.0919953475368079]))
}

# 누적 변환 계산
T = np.eye(4)
for parent, child in CHAIN:
    t, q = tf_dict[(parent, child)]
    T_i = np.eye(4)
    T_i[:3, :3] = R.from_quat(q).as_matrix()
    T_i[:3, 3] = t
    T = T @ T_i

# 최종 위치(pos)와 쿼터니언(quat) 추출
pos = T[:3, 3]
quat = R.from_matrix(T[:3, :3]).as_quat()

print('최종 위치 (map → tag36h11:4):', pos)
print('최종 쿼터니언 (map → tag36h11:4):', quat)


최종 위치 (map → tag36h11:4): [ 0.78468758 -0.11918998  0.41659849]
최종 쿼터니언 (map → tag36h11:4): [ 0.55976626 -0.53343611 -0.45475278  0.44193616]
